In [3]:
from omop_etl.datastore import DataStore
from omop_etl.load import Loader
import pandas as pd

import os
os.chdir('Z:/Covid19/Covid19_OMOP/new_pipeline/')
print(os.getcwd())

Z:\Covid19\Covid19_OMOP\new_pipeline


In [12]:
def flatten(lst):
    for el in lst:
        if isinstance(el, list):  
            # recurse
            yield from flatten(el)
        else:
            # generate
            yield el

In [5]:
omop = DataStore('config.yml')

In [6]:
with omop.connection() as con:
    dev = pd.read_sql('''
    SELECT distinct [respiratory_device]
      FROM [DWS_OMOP].[stage].[MEASUREMENT_Res_Device]
    ''', con)

In [7]:
# dev.respiratory_device.apply(lambda s: s.split(';')).to_list()

In [8]:
with omop.connection() as con:
    vmode = pd.read_sql('''
    SELECT distinct adult_vent_mode
      FROM [DWS_OMOP].[stage].[MEASUREMENT_Res_Vent]
    ''', con)

In [15]:
set(list(flatten(vmode.adult_vent_mode.apply(lambda s: s.split(';')).to_list())))

{'(S) CMV',
 'AC',
 'AC/VC+',
 'APRV',
 'APRV/Bi-Level/DuoPAP',
 'APV-CMV',
 'APV-SIMV',
 'APVcmv',
 'APVsimv',
 'ASV',
 'Auto tube compensation',
 'Bi-LEVEL',
 'CMV',
 'CPAP',
 'NIV-ST',
 'Non-Invasive',
 'Other (Comment)',
 'P-CMV',
 'P-SIMV',
 'PAV',
 'PCV',
 'PCV+',
 'PCV+ Assist',
 'PCV+ASSIST',
 'PRVC',
 'PSV',
 'SIMV',
 'SPONTANEOUS',
 'Spontaneous',
 'Stand-by',
 'VAPS',
 'VC+',
 'VS',
 'Volume Control',
 'Volume Support',
 'Volume control',
 'Volume support'}